In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
IMAGE_SIZE = 28
IMAGE_CHANNEL_NUM = 1

CONV_1_DEPTH = 32
CONV_1_SIZE = 5

CONV_2_DEPTH = 64
CONV_2_SIZE = 5

FC_SIZE = 1024

OUTPUT_SIZE = 10

DROP_OUT_RATE = 0.4
LEARNING_RATE = 0.01
BATCH_SIZE = 100
TRAINING_STEPS = 2000

In [3]:
def create_conv_layer(input_tensor, filter_depth, filter_size):
    return tf.layers.conv2d(
        inputs=input_tensor, 
        filters=filter_depth, 
        kernel_size=[filter_size, filter_size], 
        padding="same", activation=tf.nn.relu)

In [4]:
def create_max_polling_layer(input_tensor):
    return tf.layers.max_pooling2d(input_tensor, pool_size=[2, 2], strides=2)

In [5]:
def create_fc_layer(input_tensor, size, activation=tf.nn.relu):
    return tf.layers.dense(input_tensor, units=size, activation=activation)

In [6]:
def create_dropout_layer(input_tensor, rate, mode):
    return tf.layers.dropout(
        input_tensor, rate=rate, training=mode == tf.estimator.ModeKeys.TRAIN)

In [7]:
def mnist_model(features, labels, mode):
    input_tensor = tf.reshape(features["x"], [-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL_NUM])
    conv_1 = create_conv_layer(input_tensor, CONV_1_DEPTH, CONV_1_SIZE)
    pool_1 = create_max_polling_layer(conv_1)
    conv_2 = create_conv_layer(pool_1, CONV_2_DEPTH, CONV_2_SIZE)
    pool_2 = create_max_polling_layer(conv_2)
    poll_2_shape = pool_2.get_shape().as_list()
    pool_2_reshape = tf.reshape(pool_2, [-1, poll_2_shape[1] * poll_2_shape[2] * poll_2_shape[3]])
    fc = create_fc_layer(pool_2_reshape, FC_SIZE)
    dropout = create_dropout_layer(fc, DROP_OUT_RATE, mode)
    logits = create_fc_layer(dropout, OUTPUT_SIZE, activation=None)
    
    predictions = {
        "classes": tf.argmax(logits, axis=1),
        "probablities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
mnist = input_data.read_data_sets("./data")

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

In [10]:
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
mnist_classifier = tf.estimator.Estimator(model_fn=mnist_model, model_dir="./tmp/mnist_conv")
# tensors_to_log = {"probabilities": "softmax_tensor"}
tensors_to_log = {}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=BATCH_SIZE,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(input_fn=train_input_fn, steps=TRAINING_STEPS, hooks=[logging_hook])

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_conv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x116055cf8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/mnist_conv/model.ckpt.
INFO:tensorflow:
INFO:tensorflow:loss = 2.2896297, step = 1
INFO:tensorflow: (22.963 sec)
INFO:tensorflow:global_step/sec: 2.28723
INFO:tensorflow: (20.760 sec)
INFO:tensorflow:loss = 1.9594587, step = 101 (43.723 sec)
INFO:tensorflow: (21.265 sec)
INFO:tensorflow:global_step/sec: 2.32045
INFO:tensorflow